In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.ne

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-23 06:39:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-12-23 06:39:55 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [6]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("header", "true").csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(20)


+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [5]:
# Get the number of rows in the DataFrame.
print(df.count())


5331449


# Transform the Data

In [7]:
dropped_df = df.dropna()
print(dropped_df.count())

clean_df = dropped_df.dropDuplicates()
print(clean_df.count())


5330701
5330701


In [8]:
# Check the schema
clean_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Create the "review_id_table".

In [9]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = clean_df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1007YJJIFAC9J|   52769212|B009UHTGT0|     245155553| 2013-08-27|
|R100F8VKMH0GML|   22075855|B006Z96OI2|     556998710| 2015-02-28|
|R100MQXHRT2RWR|   49362036|B005PK7RW4|     475331940| 2015-06-07|
|R100Q7Y0SUHCK9|   22694784|B007UZNO4W|     240118513| 2014-08-10|
|R100QVXPVZS0DZ|   30117376|B00GZRPIEU|     206581276| 2015-01-14|
|R100UJWTF4UIOK|   43059193|B001TIOLN4|      98283682| 2013-02-12|
|R100ZA6RFJ5RUC|   10191404|B001VEB5AI|     374144182| 2012-12-05|
|R10129K7CS35V1|   24593931|B0057D81QK|     836654866| 2013-06-24|
|R101Q7XWPBH68Z|    4536548|B00CFSM670|     316045886| 2014-12-24|
|R101QDVAUUPBED|   50619433|B00020HQQE|     148453171| 2012-05-22|
|R101TUWPZXRJU3|   13146526|B000FOA7C2|       3633715| 2010-06-14|
| R101X9S0NAHJ1|   33439221|B00GXKVEJM|     833159451| 2015-08

## Create the "products" Table

In [10]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df= clean_df.select(["product_id", "product_title"])
products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B009UHTGT0|  Scrub Daddy 8 Pack|
|B006Z96OI2|Marquee 5-Piece A...|
|B005PK7RW4|Crane Usa EE-5301...|
|B007UZNO4W|Brawny Regular Ro...|
|B00GZRPIEU|Dymatize ISO100 H...|
|B001TIOLN4|Ezy Dose Disposab...|
|B001VEB5AI|Scotch™ 3M Jumbo ...|
|B0057D81QK|Futuro Open Toe K...|
|B00CFSM670|Touch Me 100% Del...|
|B00020HQQE|Nature's Way Saw ...|
|B000FOA7C2|Priva Ultra Absor...|
|B00GXKVEJM|Authentic, Mojo S...|
|B002YD7TU0|Method 8X Concent...|
|B00427NM98|GLUCOCARD VITAL M...|
|B009AH4OR4|NatureWise UltraP...|
|B003JOVR6K|Leegoal Magic Pla...|
|B00EY305CW|Omega 3 Fish Oil ...|
|B009ZJ4FSA|MRI Eo2 Edge Cola...|
|B0013OXOKU|NOW Foods - Whey ...|
|B001QRZ0LK|Odorless Garlic 1...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [11]:
customer_count=clean_df.select(["customer_id"]).count()
customer_count

5330701

In [12]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers=clean_df.groupBy("customer_id").count()
customers_df=customers.withColumnRenamed("count","customers_count")
customers_df.show(20)

+-----------+---------------+
|customer_id|customers_count|
+-----------+---------------+
|   45641495|              6|
|   48343488|              1|
|   17171509|             18|
|    7255220|              2|
|   10262401|              2|
|   32166090|              5|
|   14011071|              5|
|   37825239|              8|
|   29666921|             16|
|   14242735|              2|
|   29265130|              4|
|   51379647|             26|
|   46230753|              1|
|   24831853|              1|
|   13519509|             10|
|   28058398|              1|
|   10086411|              2|
|   45303863|              1|
|   34981117|             11|
|   12439599|              3|
+-----------+---------------+
only showing top 20 rows



## Create the "vine_table".

In [13]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1007YJJIFAC9J|          5|            0|          0|   N|
|R100F8VKMH0GML|          5|            0|          0|   N|
|R100MQXHRT2RWR|          1|            0|          0|   N|
|R100Q7Y0SUHCK9|          4|            0|          0|   N|
|R100QVXPVZS0DZ|          5|            0|          0|   N|
|R100UJWTF4UIOK|          5|            0|          0|   N|
|R100ZA6RFJ5RUC|          4|            0|          0|   N|
|R10129K7CS35V1|          3|            0|          1|   N|
|R101Q7XWPBH68Z|          5|            0|          0|   N|
|R101QDVAUUPBED|          3|            0|          0|   N|
|R101TUWPZXRJU3|          5|            6|          7|   N|
| R101X9S0NAHJ1|          5|            0|          0|   N|
|R1022ZJ6W5AZCP|          5|            0|          0|   N|
|R1027FWHK77EX1|          3|            

# Load

In [14]:
mode = "overwrite"
jdbc_url="jdbc:postgresql://database-big-data.cteomnge5i1n.us-east-1.rds.amazonaws.com:5432/health"
config = {"user":"", "password": "", "driver":"org.postgresql.Driver"}

In [15]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [19]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [17]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [18]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)